## IPEDS Data Exploration and Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

## Read in School Information

In [ ]:
directory = pd.read_csv("../data/ipeds_directory_info_2017_18.csv")

In [ ]:
directory.head()

In [ ]:
## look at shape
directory.shape

In [ ]:
directory.tail()

In [ ]:
## count of nulls
directory.isna().sum()

## Read in Admissions Information

In [ ]:
admissions_cond = pd.read_csv("../data/ipeds_admissions_2017_18.csv")

In [ ]:
admissions_cond.head()

In [ ]:
## How many options in entrance considerations? How many schools stress each the most out of 7153 schools.
admissions_cond['cond_gpa'].value_counts()

In [ ]:
admissions_cond['cond_test_scores'].value_counts()

In [ ]:
admissions_cond['cond_college_prep'].value_counts()

## Added columns for sat_comp_25 and sat_comp_75  as these were not in database

In [ ]:
admissions_cond['sat_comp_25'] = admissions_cond['sat_verb_25'] + admissions_cond['sat_math_25']
admissions_cond['sat_comp_75'] = admissions_cond['sat_verb_75'] + admissions_cond['sat_math_75']

## Some EDA on this dataset

In [ ]:
## look at shape
admissions_cond.shape

In [ ]:
## look at nulls
admissions_cond.isna().sum()

## Read in derived admissions stats

In [ ]:
admissions_derived = pd.read_csv("../data/ipeds_derived_admiss_stats_2017.csv")

In [ ]:
admissions_derived.head()

In [ ]:
admissions_derived.shape

In [ ]:
## Histgram of percent admitted
plt.figure(figsize=(12, 8))
admissions_derived['perc_admit_total'].hist(bins=20)
plt.title('Percentage of Applicants Accepted')

In [ ]:
## Histgram of percent enrolled
plt.figure(figsize=(12, 8))
admissions_derived['admit_yield_total'].hist(bins=20)
plt.title('Percentage of Students that Enrolled')

In [ ]:
##Scatter Plot of Total Students accepted vs. enrolled
plt.figure(figsize=(12, 8))
plt.scatter('perc_admit_total', 'admit_yield_total', data = admissions_derived, c='orange')
plt.xlabel('% Total Admitted')
plt.ylabel('% Total Enrolled')
plt.title('Percentage of Students Accepted vs. Enrolled', color = 'black');

In [ ]:
##Scatter Plot of Women accepted vs. enrolled
plt.figure(figsize=(12, 8))
plt.scatter('perc_admit_women', 'admit_yield_women', data = admissions_derived, c='palevioletred')
plt.xlabel('% Women Admitted')
plt.ylabel('% Women Enrolled')
plt.title('Percentage of Women Accepted vs. Enrolled', color = 'black');

In [ ]:
##Scatter Plot of Men accepted vs. enrolled
plt.figure(figsize=(12, 8))
plt.scatter('perc_admit_men', 'admit_yield_men', data = admissions_derived)
plt.xlabel('% Men Admitted')
plt.ylabel('% Men Enrolled')
plt.title('Percentage of Men Accepted vs. Enrolled', color = 'black');

## Schools that accept less than 35% of applicants...

In [ ]:
## Some shools had zero acceptance so I filtered these out.
hi_selective_schools = admissions_derived.loc[(admissions_derived['perc_admit_total'] < 35) & (admissions_derived['perc_admit_total'] != 0)]

In [ ]:
hi_selective_schools.shape

In [ ]:
hi_selective_schools

In [ ]:
hi_selective_schools.tail()

## Merge with basic school directory info so we have school name and loction...

In [ ]:
## create subset of directory dataframe
basic_school_info = directory[['school_id', 'institution_name', 'city', 'state_abbr', 'zipcode']]

In [ ]:
basic_school_info

In [ ]:
## merge with hi_selective_schools
hi_selective_schools = pd.merge(basic_school_info, hi_selective_schools, how = 'right', on = 'school_id')

In [ ]:
hi_selective_schools.head()

In [ ]:
hi_selective_schools.tail()

In [ ]:
hi_selective_schools.shape

## Website used for highly selective school criteria. Used second category to pull first two groups of schools.
https://www.collegetransitions.com/college-selectivity/#:~:text=EXTREMELY%20SELECTIVE%20colleges%20accept%20fewer,score%20of%20at%20least%201340.

## Define "highly selective" as all schools that meet these conditions: 
- Accept fewer than 35% of all applicants,
- AND possess an average composite ACT (25%tile) score of at least 29 
- OR an average combined SAT (25%tile) score of at least 1340.

In [ ]:
test_scores = admissions_cond[['school_id', 'sat_comp_25', 'sat_comp_75','sat_verb_25', 'sat_verb_75', 'sat_math_25', 'sat_math_75', 
                              'act_comp_25', 'act_comp_75', 'act_eng_25', 'act_eng_75', 'act_math_25', 'act_math_75']]

### Merge in the test score data

In [ ]:
hi_selective_schools = pd.merge(hi_selective_schools, test_scores, how = 'left', on = 'school_id')

In [ ]:
hi_selective_schools.head()

In [ ]:
hi_selective_schools.isna().sum()

## Now filter for the test score criteria

In [ ]:
hi_schools_with_scores = hi_selective_schools.loc[(hi_selective_schools['act_comp_25'] >= 29.0)|(hi_selective_schools['sat_comp_25'] >= 1340.0)]

In [ ]:
hi_schools_with_scores.shape

In [ ]:
hi_schools_with_scores.head()

In [ ]:
hi_schools_with_scores.reset_index(drop=True)

In [ ]:
hi_schools_with_scores.sort_values(by = ['perc_admit_total'])

In [ ]:
## Average act_comp_25
hi_schools_with_scores['act_comp_25'].mean()

In [ ]:
## Average act_comp_75
hi_schools_with_scores['act_comp_75'].mean()

In [ ]:
## Average sat_comp_25
hi_schools_with_scores['sat_comp_25'].mean()

In [ ]:
## Average sat_comp_75
hi_schools_with_scores['sat_comp_75'].mean()